<a href="https://colab.research.google.com/github/geovatatsuga/dashboard-e-previsao-churn/blob/main/Projeto_de_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalando o pyodbc,dash e plotly para conexão com SQL server

In [1]:
pip install pyodbc plotly dash dash-bootstrap-components matplotlib seaborn scikit-learn joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.2/346.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.4/202.4 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [ ]:
# Importando todas as bibliotecas necessarias para o projeto

In [2]:
import pyodbc
import pandas as pd
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

import pyodbc
import pandas as pd
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report, confusion_matrix

from sklearn.preprocessing import LabelEncoder

import joblib

from sklearn.metrics import classification_report, confusion_matrix

from sklearn.preprocessing import LabelEncoder

import joblib


In [ ]:
# Essa parte do codigo funcionará apenas se o SQL Server estiver ativo, não sendo necessario rodar essa cedula, uma vez que já foram realizadas as transformações do pytohn para o sql

In [ ]:
# (Não necessario rodar) Serão criadas agora colunas essencias necessarias para analises posteriores

In [ ]:
# Adicionando a coluna Status_do_Churn
dados_conexao = (
    'Driver={ODBC Driver 18 for SQL Server};'
    'Server=DESKTOP-RMJHLGK;'
    'Database=db_Churn;'
    'Trusted_Connection=yes;'  # Usa a autenticação do Windows
    'TrustServerCertificate=yes;' ) # Ignora a validação do certificado SSL)
conexao = pyodbc.connect(dados_conexao)
print("Conexão bem-sucedida!")
cursor = conexao.cursor()
cursor.execute("ALTER TABLE prod_Churn ADD Status_do_Churn INT;")
conexao.commit()

# Atualizando os valores de Status_do_Churn
cursor.execute("""
    UPDATE prod_Churn
    SET Status_do_Churn = CASE
        WHEN Status_do_Cliente = 'Churned' THEN 1
        ELSE 0
    END;
""")
conexao.commit()

cursor.execute("ALTER TABLE prod_Churn ADD Faixa_Carga_Mensal VARCHAR(10);")
conexao.commit()
print("Coluna 'Faixa_Carga_Mensal' adicionada com sucesso.")

# Preenchendo os valores na nova coluna
cursor.execute("""
    UPDATE prod_Churn
    SET Faixa_Carga_Mensal = CASE
        WHEN Carga_Mensal < 20 THEN '< 20'
        WHEN Carga_Mensal < 50 THEN '20-50'
        WHEN Carga_Mensal < 100 THEN '50-100'
        ELSE '> 100'
    END;
""")
conexao.commit()
print("Valores de 'Faixa_Carga_Mensal' atualizados com sucesso.")

cursor.execute("ALTER TABLE prod_Churn ADD Faixa_Idade VARCHAR(10);")
conexao.commit()
print("Coluna 'Faixa_Idade' adicionada com sucesso.")

# Preenchendo os valores na nova coluna
cursor.execute("""
   UPDATE prod_Churn
    SET Faixa_Idade = CASE
        WHEN Idade < 20 THEN '< 20'
        WHEN Idade < 36 THEN '20-35'
        WHEN Idade < 51 THEN '36-50'
    ELSE '> 50'
 END;
""")
conexao.commit()
print("Valores de 'Faixa_Idade' atualizados com sucesso.")

# Adicionando a coluna Faixa_Tempo_Contrato
cursor.execute("ALTER TABLE prod_Churn ADD Faixa_Tempo_Contrato VARCHAR(15);")
conexao.commit()
print("Coluna 'Faixa_Tempo_Contrato' adicionada com sucesso.")

# Preenchendo os valores na nova coluna
cursor.execute("""
    UPDATE prod_Churn
    SET Faixa_Tempo_Contrato = CASE
        WHEN Tempo_de_Contrato_em_Meses < 6 THEN '< 6 Meses'
        WHEN Tempo_de_Contrato_em_Meses < 12 THEN '6-12 Meses'
        WHEN Tempo_de_Contrato_em_Meses < 18 THEN '12-18 Meses'
        WHEN Tempo_de_Contrato_em_Meses < 24 THEN '18-24 Meses'
        ELSE '>= 24 Meses'
    END;
""")
conexao.commit()
print("Valores de 'Faixa_Tempo_Contrato' atualizados com sucesso.")
query = "SELECT * FROM dbo.prod_Churn"
df = pd.read_sql(query, conexao)

# Salvar resultado
df.to_csv("df_final", index=False)

# Fechando cursor e conexão -----------------------------------------------------
cursor.close()
conexao.close()

In [ ]:
import numpy as np

# 1. Status do Churn (se cliente é churn)
df['Status_do_Churn'] = (df['Status_do_Cliente'] == 'Churned').astype(int)

# 2. Faixa de Cobrança Mensal (baseado em Cobrança_Mensal)
df['Faixa_Carga_Mensal'] = np.select(
    condlist=[
        df['Cobrança_Mensal'] < 20,
        df['Cobrança_Mensal'] < 50,
        df['Cobrança_Mensal'] < 100
    ],
    choicelist=['< 20', '20-50', '50-100'],
    default='> 100'
)

# 3. Faixa de Idade (baseado em Idade)
df['Faixa_Idade'] = np.select(
    condlist=[
        df['Idade'] < 20,
        df['Idade'] < 36,
        df['Idade'] < 51
    ],
    choicelist=['< 20', '20-35', '36-50'],
    default='> 50'
)

# 4. Faixa de Tempo de Contrato (baseado em Tempo_de_Contrato_Meses)
df['Faixa_Tempo_Contrato'] = np.select(
    condlist=[
        df['Tempo_de_Contrato_Meses'] < 6,
        df['Tempo_de_Contrato_Meses'] < 12,
        df['Tempo_de_Contrato_Meses'] < 18,
        df['Tempo_de_Contrato_Meses'] < 24
    ],
    choicelist=['< 6 Meses', '6-12 Meses', '12-18 Meses', '18-24 Meses'],
    default='>= 24 Meses'
)
df.to_csv('df_atualizado.csv', index=False)


NameError: name 'df' is not defined

In [3]:
caminho = 'df_atualizado.csv'
df = pd.read_csv(caminho)
df

,Customer_ID,Gênero,Idade,Casado,Estado,Número_de_Indicações,Tempo_de_Contrato_Meses,Promoção,Serviço_Telefônico,Múltiplas_Linhas,...,Cobranças_Extras_de_Dados,Cobranças_de_Ligações_Longas,Receita_Total,Status_do_Cliente,Categoria_de_Churn,Motivo_de_Churn,Status_do_Churn,Faixa_Carga_Mensal,Faixa_Idade,Faixa_Tempo_Contrato
0,19877-DEL,Masculino,35,Não,Delhi,7,27,NaN,Sim,Não,...,0,381.51,974.81,Permaneceu,NaN,NaN,0,50-100,20-35,>= 24 Meses
1,58353-MAH,Feminino,45,Sim,Maharashtra,14,13,NaN,Sim,Sim,...,10,96.21,610.28,Permaneceu,NaN,NaN,0,< 20,36-50,12-18 Meses
2,25063-WES,Masculino,51,Não,West Bengal,4,35,Promoção 5,Sim,Não,...,0,134.60,415.45,Churned,Concorrente,Concorrente ofereceu dispositivos melhores,1,50-100,> 50,>= 24 Meses
3,59787-KAR,Masculino,79,Não,Karnataka,3,21,Promoção 4,Sim,Não,...,0,361.66,1599.51,Churned,Insatisfação,Insatisfação com o produto,1,50-100,> 50,18-24 Meses
4,28544-TAM,Feminino,80,Não,Tamil Nadu,3,8,NaN,Sim,Não,...,0,22.14,289.54,Churned,Insatisfação,Problemas com rede,1,50-100,> 50,6-12 Meses
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6413,43397-UTT,Feminino,41,Não,Uttar Pradesh,6,33,NaN,Sim,Sim,...,0,37.96,934.71,Churned,Concorrente,Concorrente fez oferta melhor,1,50-100,36-50,>= 24 Meses
6414,22240-HAR,Masculino,51,Sim,Haryana,0,19,Promoção 1,Não,NaN,...,0,0.00,3139.80,Permaneceu,NaN,NaN,0,20-50,> 50,18-24 Meses
6415,78682-KER,Feminino,65,Sim,Kerala,14,25,NaN,Sim,Sim,...,0,599.64,3469.49,Churned,Insatisfação,Insatisfação com o produto,1,50-100,> 50,>= 24 Meses
6416,17816-ODI,Masculino,23,Não,Odisha,2,10,NaN,Sim,Sim,...,0,2107.52,6588.22,Permaneceu,NaN,NaN,0,50-100,20-35,6-12 Meses


In [4]:
#Criando dfs para facilitar a visualização no dashboard

# Cálculo de métricas gerais
total_clientes = len(df)
novos_clientes = len(df[df['Status_do_Cliente'] == 'Novo Cliente'])
total_churn = len(df[df['Status_do_Cliente'] == 'Churned'])
taxa_churn_geral = round((total_churn / total_clientes) * 100, 2)

print("Total clientes:", total_clientes)
print("Novos clientes:", novos_clientes)
print("Total churn:", total_churn)
print("Taxa churn geral (%):", taxa_churn_geral)

contagem_churn_demografico = df[df['Status_do_Churn'] == 1].groupby('Gênero').size().reset_index(name='Total de Churn')

taxa_churn_por_tipo_pagamento = df.groupby('Método_de_Pagamento')['Status_do_Churn'].mean() * 100
taxa_churn_por_tipo_pagamento = taxa_churn_por_tipo_pagamento.round(2).sort_values(ascending=False)

taxa_churn_por_contrato = df.groupby('Contrato')['Status_do_Churn'].mean() * 100
taxa_churn_por_contrato = taxa_churn_por_contrato.round(2).sort_values(ascending=False)

taxa_churn_estados_top5 = df.groupby('Estado')['Status_do_Churn'].mean().nlargest(5) * 100
taxa_churn_estados_top5 = taxa_churn_estados_top5.round(2)

total_churn_categoria = df.groupby('Categoria_de_Churn')['Status_do_Churn'].sum().sort_values(ascending=False)

total_churn_tipo_internet = df.groupby('Tipo_de_Internet')['Status_do_Churn'].sum().sort_values(ascending=False)

Total clientes: 6418
Novos clientes: 411
Total churn: 1732
Taxa churn geral (%): 26.99


In [5]:
# Definir a ordem correta das faixas etárias
ordem_correta = ['< 20', '20-35', '36-50', '> 50']

# Agrupar e contar o número de clientes por faixa etária
total_cliente_grupo_idade = df.groupby('Faixa_Idade').size().reset_index(name='Total_Clientes')

# Converter a coluna 'Faixa_Idade' para categórica com ordem definida
total_cliente_grupo_idade['Faixa_Idade'] = pd.Categorical(total_cliente_grupo_idade['Faixa_Idade'],
                                                          categories=ordem_correta,
                                                          ordered=True)

# Ordenar pela coluna categorizada
total_cliente_grupo_idade = total_cliente_grupo_idade.sort_values('Faixa_Idade')


taxa_churn_grupo_idade = df.groupby('Faixa_Idade')['Status_do_Churn'].mean() * 100
taxa_churn_grupo_idade = taxa_churn_grupo_idade.round(2)

# Criar a categorização com ordem definida
taxa_churn_grupo_idade.index = pd.Categorical(taxa_churn_grupo_idade.index, categories=ordem_correta, ordered=True)

# Ordenar pelo índice categórico
taxa_churn_grupo_idade = taxa_churn_grupo_idade.sort_index().round(2)




In [6]:
# Definir a ordem correta das faixas de tempo contratado (somente em Meses)
ordem_correta = ['< 6 Meses', '6-12 Meses', '12-18 Meses', '18-24 Meses', '>= 24 Meses']

# Agrupar e contar o número de clientes por faixa de tempo contratado
Total_clientes_por_tempo_contratado = df.groupby('Faixa_Tempo_Contrato').size().reset_index(name='Total_Clientes')

# Converter a coluna 'Faixa_Tempo_Contrato' para categórica com ordem definida
Total_clientes_por_tempo_contratado['Faixa_Tempo_Contrato'] = pd.Categorical(
    Total_clientes_por_tempo_contratado['Faixa_Tempo_Contrato'],
    categories=ordem_correta,
    ordered=True
)

# Ordenar pela coluna categorizada
Total_clientes_por_tempo_contratado = Total_clientes_por_tempo_contratado.sort_values('Faixa_Tempo_Contrato')



# Calcular a taxa de churn por faixa de tempo contratado
taxa_churn_por_tempo_contratado = df.groupby('Faixa_Tempo_Contrato')['Status_do_Churn'].mean() * 100

# Converter o índice para categórico com ordem definida
taxa_churn_por_tempo_contratado.index = pd.Categorical(
    taxa_churn_por_tempo_contratado.index,
    categories=ordem_correta,
    ordered=True
)

# Ordenar pelo índice categórico
taxa_churn_por_tempo_contratado = taxa_churn_por_tempo_contratado.sort_index().round(2)


In [ ]:
import dash_bootstrap_components as dbc
from dash import dcc, html, Dash, Input, Output
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# ================================
# CONSTANTES DE CORES
# ================================
COR_PRIMARIA = "#8e44ad"
COR_SECUNDARIA = "#e74c3c"
COR_SUCESSO = "#27ae60"
COR_ALERTA = "#c0392b"
COR_AVISO = "#f39c12"
FUNDO_ESCURO = "#1a1a1a"
TEXTO_CLARO = "#ffffff"
SLOT_CLARO = "#ffffff"

todos_estados = pd.Series({
    'Uttar Pradesh': 629, 'Tamil Nadu': 600, 'Maharashtra': 504,
    'Karnataka': 470, 'Haryana': 398, 'Andhra Pradesh': 395,
    'West Bengal': 368, 'Punjab': 342, 'Bihar': 336, 'Gujarat': 335,
    'Jammu & Kashmir': 320, 'Madhya Pradesh': 288, 'Telangana': 281,
    'Rajasthan': 259, 'Kerala': 200, 'Odisha': 152, 'Assam': 139,
    'Delhi': 127, 'Jharkhand': 113, 'Uttarakhand': 62,
    'Chhattisgarh': 59, 'Puducherry': 41
})

# ================================
# FUNÇÕES AUXILIARES
# ================================
def criar_card(valor, titulo, cor_fundo, icone, card_id):
    return dbc.Card(
        dbc.CardBody([
            html.Div(className="d-flex align-items-center", children=[
                html.Div(className="icon-container me-3", children=[
                    html.I(className=icone, style={"fontSize": "2.5rem", "color": cor_fundo})
                ]),
                html.Div([
                    html.H2(valor, className="card-value mb-0", style={'color': cor_fundo}),
                    html.P(titulo, className="card-title mb-0", style={'color': '#555555'})
                ])
            ])
        ]),
        className="shadow-lg metric-card",
        style={
            'borderRadius': '15px',
            'backgroundColor': SLOT_CLARO,
            'transition': 'transform 0.2s',
            'margin': '0.5rem',
            'border': '1px solid #e0e0e0'
        },
        id=card_id
    )

def criar_grafico_combinado(total_df, taxa_series, titulo, eixo_x):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(
        go.Bar(
            x=total_df.iloc[:, 0],
            y=total_df['Total_Clientes'],
            name='Total de Clientes',
            marker_color=COR_PRIMARIA,
            opacity=0.7,
            text=total_df['Total_Clientes'],
            textposition='outside',
            texttemplate='%{text:,}',
            textfont=dict(color=TEXTO_CLARO)
        ),
        secondary_y=False
    )
    fig.add_trace(
        go.Scatter(
            x=taxa_series.index,
            y=taxa_series.values,
            name='Taxa de Churn',
            line=dict(color='red', width=2),
            mode='lines+markers+text',
            marker=dict(size=6, symbol='diamond'),
            text=[f'{val:.1f}%' for val in taxa_series.values],
            textposition='top center',
            textfont=dict(color=TEXTO_CLARO)
        ),
        secondary_y=True
    )
    max_churn = max(taxa_series.values)
    max_tick = 10 * ((int(max_churn) // 10 + 1))
    fig.update_layout(
        title_text=titulo,
        template='plotly_dark',
        height=320,
        margin=dict(l=15, r=15, t=60, b=20),
        legend=dict(orientation="h", yanchor="bottom", y=1.02),
        xaxis_title=eixo_x,
        xaxis=dict(showgrid=False, color=TEXTO_CLARO),
        yaxis=dict(showgrid=False, color=TEXTO_CLARO),
        title_font=dict(size=18, family='Arial', color=TEXTO_CLARO),
        title_x=0.5,
        paper_bgcolor=FUNDO_ESCURO,
        plot_bgcolor=FUNDO_ESCURO
    )
    fig.update_yaxes(title_text="Clientes", secondary_y=False, showgrid=False)
    fig.update_yaxes(title_text="Churn (%)", secondary_y=True,
                   range=[0, max_tick], showgrid=False)
    return fig

def criar_grafico_horizontal(data_series, titulo, label_y, label_x, formato_percent=False):
    if formato_percent:
        text_template = '%{text:.1f}%'
        tick_format = '.1f%'
    else:
        text_template = '%{text:,}'
        tick_format = ','
    fig = px.bar(
        y=[str(i) for i in data_series.index],
        x=data_series.values,
        orientation='h',
        title=titulo,
        labels={'y': label_y, 'x': label_x},
        color_discrete_sequence=[COR_PRIMARIA]
    )
    fig.update_traces(
        text=data_series.values,
        textposition='outside',
        texttemplate=text_template,
        textfont=dict(color=TEXTO_CLARO)
    )
    fig.update_layout(
        template='plotly_dark',
        height=280,
        margin=dict(l=15, r=15, t=60, b=20),
        xaxis=dict(
            showgrid=False,
            tickformat=tick_format,
            range=[0, data_series.max() * 1.15],
            color=TEXTO_CLARO
        ),
        yaxis=dict(showgrid=False, color=TEXTO_CLARO),
        title_font=dict(size=16, family='Arial', color=TEXTO_CLARO),
        title_x=0.5,
        paper_bgcolor=FUNDO_ESCURO,
        plot_bgcolor=FUNDO_ESCURO
    )
    return fig

def criar_mapa_churn():
    # Preparação dos dados
    df_map = todos_estados.reset_index()
    df_map.columns = ['Estado', 'Churn']
    df_map['Estado'] = df_map['Estado'].replace({
        'Jammu & Kashmir': 'Jammu and Kashmir',
        'Andhra Pradesh': 'Andhra Pradesh',
        'Uttar Pradesh': 'Uttar Pradesh'
    })

    # Configuração do mapa
    geojson_url = "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"

    fig = px.choropleth_mapbox(
        df_map,
        geojson=geojson_url,
        locations='Estado',
        featureidkey='properties.ST_NM',
        color='Churn',
        color_continuous_scale='Viridis',  # Escala de cor moderna
        range_color=(0, todos_estados.max()),
        mapbox_style="carto-darkmatter",
        zoom=3.5,
        center={"lat": 22.9734, "lon": 78.6569},
        opacity=0.8,
        hover_data={'Estado': True, 'Churn': ':,.0f'},
        labels={'Churn': 'Casos de Churn'}
    )

    # === Adição do Título Estilizado ===
    fig.update_layout(
        title={
            'text': "<b>Distribuição de Churn por Estado</b>",
            'font': {'size': 24, 'color': TEXTO_CLARO, 'family': 'Arial Black'},
            'x': 0.5,
            'y': 0.95,
            'xanchor': 'center',
            'yanchor': 'top',
            'pad': {'b': 20}
        },
        margin={"r":20,"t":80,"l":20,"b":20},
        coloraxis_colorbar=dict(
            title='Casos de Churn',
            thickness=15,
            len=0.6,
            title_font=dict(color=TEXTO_CLARO, size=12),
            tickfont=dict(color=TEXTO_CLARO, size=10),
            orientation='v',
            yanchor='middle',
            y=0.5,
            x=0.93
        ),
        paper_bgcolor=FUNDO_ESCURO,
        plot_bgcolor=FUNDO_ESCURO
    )

    # === Configurações de Zoom ===
    fig.update_geos(
        visible=False,
        bgcolor='rgba(0,0,0,0)',
        lakecolor='#2a2a2a',
        landcolor='#1a1a1a',
        showcountries=True,
        countrycolor='#444444',
        countrywidth=0.5
    )

    # === Hover Interativo ===
    fig.update_traces(
        hovertemplate="<b>%{customdata[0]}</b><br>" +
                      "Churn: %{customdata[1]}<extra></extra>",
        marker=dict(line=dict(width=0.5, color='#333333'))
    )
    mapbox_layers=[
            {
                "sourcetype": "geojson",
                "source": geojson_url,
                "type": "symbol",
                "text-field": ["get", "ST_NM"],
                "text-size": 12,
                "text-color": "white",
                "text-halo-color": "#2a2a2a",
                "text-halo-width": 1
            }
        ]

    return fig

# ================================
# COMPONENTES PRINCIPAIS
# ================================
cabecalho_e_metricas = [
    dbc.Row(
        dbc.Col(
            html.Div([
                html.Div([
                    html.I(className="fas fa-chart-line me-2",
                          style={"color": COR_PRIMARIA, "fontSize": "2.5rem"}),
                    html.H1("Dashboard de Churn",
                           className="display-4 mb-0",
                           style={'color': TEXTO_CLARO})
                ], className="d-flex justify-content-center align-items-center"),
                html.P("Análise de Retenção de Clientes",
                      className="lead",
                      style={'color': COR_PRIMARIA})
            ], className="text-center py-4"),
            width=12
        )
    ),
    dbc.Row([
        dbc.Col(criar_card(
            f"{total_clientes:,}",
            "Total Clientes",
            COR_PRIMARIA,
            "fas fa-users",
            {'type': 'metric-card', 'index': 0}
        ), width=3),
        dbc.Col(criar_card(
            f"{novos_clientes:,}",
            "Novos Clientes",
            COR_SUCESSO,
            "fas fa-user-plus",
            {'type': 'metric-card', 'index': 1}
        ), width=3),
        dbc.Col(criar_card(
            f"{total_churn:,}",
            "Clientes Perdidos",
            COR_ALERTA,
            "fas fa-user-minus",
            {'type': 'metric-card', 'index': 2}
        ), width=3),
        dbc.Col(criar_card(
            f"{taxa_churn_geral:.1f}%",
            "Taxa de Churn",
            COR_AVISO,
            "fas fa-percentage",
            {'type': 'metric-card', 'index': 3}
        ), width=3)
    ], className="g-4 mb-4")
]

# ================================
# CRIAÇÃO DOS GRÁFICOS
# ================================
fig_faixa_etaria = criar_grafico_combinado(
    total_cliente_grupo_idade,
    taxa_churn_grupo_idade,
    "Churn por Faixa Etária",
    "Faixa Etária"
)
fig_tempo_contrato = criar_grafico_combinado(
    Total_clientes_por_tempo_contratado,
    taxa_churn_por_tempo_contratado,
    "Churn por Tempo de Contrato",
    "Meses de Contrato"
)

fig_churn_genero = px.pie(
    names=contagem_churn_demografico['Gênero'],
    values=contagem_churn_demografico['Total de Churn'],
    title='Churn por Gênero',
    color_discrete_sequence=[COR_PRIMARIA, COR_SECUNDARIA]
)

legenda_genero = html.Div([
    html.Table(
        className="table",
        children=[
            html.Thead(html.Tr([
                html.Th("Cor"),
                html.Th("Gênero"),
                html.Th("Clientes")
            ])),
            html.Tbody([
                html.Tr([
                    html.Td(html.Div(style={'width': '20px', 'height': '20px', 'backgroundColor': COR_PRIMARIA})),
                    html.Td("Feminino"),
                    html.Td(f"{contagem_churn_demografico[contagem_churn_demografico['Gênero'] == 'Feminino']['Total de Churn'].values[0]:,}")
                ]),
                html.Tr([
                    html.Td(html.Div(style={'width': '20px', 'height': '20px', 'backgroundColor': COR_SECUNDARIA})),
                    html.Td("Masculino"),
                    html.Td(f"{contagem_churn_demografico[contagem_churn_demografico['Gênero'] == 'Masculino']['Total de Churn'].values[0]:,}")
                ])
            ])
        ],
        style={
            'marginTop': '20px',
            'backgroundColor': FUNDO_ESCURO,
            'color': TEXTO_CLARO,
            'width': '100%'
        }
    )
])

fig_churn_genero.update_traces(
    textinfo='label+percent+value',
    texttemplate='%{label}<br>%{value} (%{percent})',
    marker=dict(line=dict(color='black', width=1)),
    textfont=dict(color=TEXTO_CLARO, size=14),
    hovertemplate="<b>%{label}</b><br>Quantidade: %{value}<br>Porcentagem: %{percent}<extra></extra>",
    textposition='auto',
    insidetextorientation='horizontal'
)
fig_churn_genero.update_layout(
    template='plotly_dark',
    height=320,
    margin=dict(l=15, r=15, t=60, b=20),
    title_font=dict(size=18, family='Arial', color=TEXTO_CLARO),
    title_x=0.5,
    paper_bgcolor=FUNDO_ESCURO,
    showlegend=False
)

fig_taxa_pagamento = criar_grafico_horizontal(
    taxa_churn_por_tipo_pagamento,
    'Taxa de Churn por Método de Pagamento',
    'Método de Pagamento',
    'Taxa de Churn (%)',
    formato_percent=True
)
fig_churn_contrato = criar_grafico_horizontal(
    taxa_churn_por_contrato,
    'Taxa de Churn por Tipo de Contrato',
    'Tipo de Contrato',
    'Taxa de Churn (%)',
    formato_percent=True
)
fig_estado = criar_grafico_horizontal(
    todos_estados.sort_values(ascending=False).head(5),
    'Taxa de Churn por Estado (Top 5)',
    'Estado',
    'Taxa de Churn (%)',
    formato_percent=False
)
fig_categoria = criar_grafico_horizontal(
    total_churn_categoria,
    'Total de Churn por Categoria',
    'Categoria',
    'Clientes',
    formato_percent=False
)
fig_internet = criar_grafico_horizontal(
    total_churn_tipo_internet,
    'Total de Churn por Tipo de Internet',
    'Tipo de Internet',
    'Clientes',
    formato_percent=False
)

# ================================
# LAYOUT PRINCIPAL COM ABAS
# ================================
app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc.icons.FONT_AWESOME])
app.layout = dbc.Container([
    dbc.Tabs([
        dbc.Tab(
            label="Dashboard Principal",
            children=[
                *cabecalho_e_metricas,
                dbc.Row([
                    dbc.Col(
                        dbc.Card(
                            html.Div([
                                dcc.Graph(figure=fig_churn_genero),
                                legenda_genero
                            ]),
                            style={'backgroundColor': FUNDO_ESCURO, 'border': '1px solid #333'}
                        ), width=4
                    ),
                    dbc.Col(
                        dbc.Card(
                            dcc.Graph(figure=fig_faixa_etaria),
                            style={'backgroundColor': FUNDO_ESCURO, 'border': '1px solid #333'}
                        ), width=4
                    ),
                    dbc.Col(
                        dbc.Card(
                            dcc.Graph(figure=fig_tempo_contrato),
                            style={'backgroundColor': FUNDO_ESCURO, 'border': '1px solid #333'}
                        ), width=4
                    )
                ], className="mb-4"),
                dbc.Row([
                    dbc.Col(
                        dbc.Card(
                            dcc.Graph(figure=fig_taxa_pagamento),
                            style={'backgroundColor': FUNDO_ESCURO, 'border': '1px solid #333'}
                        ), width=3
                    ),
                    dbc.Col(
                        dbc.Card(
                            dcc.Graph(figure=fig_churn_contrato),
                            style={'backgroundColor': FUNDO_ESCURO, 'border': '1px solid #333'}
                        ), width=3
                    ),
                    dbc.Col(
                        dbc.Card(
                            dcc.Graph(figure=fig_estado),
                            style={'backgroundColor': FUNDO_ESCURO, 'border': '1px solid #333'}
                        ), width=3
                    ),
                    dbc.Col(
                        dbc.Card(
                            dcc.Graph(figure=fig_categoria),
                            style={'backgroundColor': FUNDO_ESCURO, 'border': '1px solid #333'}
                        ), width=3
                    ),
                    dbc.Col(
                        dbc.Card(
                            dcc.Graph(figure=fig_internet),
                            style={'backgroundColor': FUNDO_ESCURO, 'border': '1px solid #333'}
                        ), width=3
                    )
                ], className="mb-4 g-2", justify="around")
            ],
            tabClassName="fw-bold"
        ),
        dbc.Tab(
            label="Mapa de Churn",
            children=[
                dbc.Row([
                    dbc.Col(
                        dbc.Card(
                            dcc.Graph(
                                id='mapa-churn',
                                figure=criar_mapa_churn(),
                                config={
        'displayModeBar': True,
        'scrollZoom': True,
        'modeBarButtonsToAdd': ['zoomInMapbox', 'zoomOutMapbox']
    }
                            ),
                            style={
                                'border': f'2px solid {COR_PRIMARIA}',
                                'borderRadius': '15px',
                                'backgroundColor': FUNDO_ESCURO
                            }
                        ), width=12
                    )
                ], className="mt-4")
            ],
            tabClassName="fw-bold"
        )
    ])
], fluid=True, style={
    'backgroundColor': FUNDO_ESCURO,
    'padding': '20px',
    'minHeight': '100vh'
})

if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>

In [7]:
data = df.copy()

In [ ]:
df.columns

Index(['Customer_ID', 'Gênero', 'Idade', 'Casado', 'Estado',
       'Número_de_Indicações', 'Tempo_de_Contrato_Meses', 'Promoção',
       'Serviço_Telefônico', 'Múltiplas_Linhas', 'Serviço_de_Internet',
       'Tipo_de_Internet', 'Segurança_Online', 'Backup_Online',
       'Proteção_de_Dispositivo', 'Suporte_Premium', 'TV_Streaming',
       'Filmes_Streaming', 'Música_Streaming', 'Dados_Ilimitados', 'Contrato',
       'Fatura_Digital', 'Método_de_Pagamento', 'Cobrança_Mensal',
       'Total_Cobrado', 'Total_Reembolsos', 'Cobranças_Extras_de_Dados',
       'Cobranças_de_Ligações_Longas', 'Receita_Total', 'Status_do_Cliente',
       'Categoria_de_Churn', 'Motivo_de_Churn', 'Status_do_Churn',
       'Faixa_Carga_Mensal', 'Faixa_Idade', 'Faixa_Tempo_Contrato'],
      dtype='object')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6418 entries, 0 to 6417
Data columns (total 36 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Customer_ID                   6418 non-null   object 
 1   Gênero                        6418 non-null   object 
 2   Idade                         6418 non-null   int64  
 3   Casado                        6418 non-null   object 
 4   Estado                        6418 non-null   object 
 5   Número_de_Indicações          6418 non-null   int64  
 6   Tempo_de_Contrato_Meses       6418 non-null   int64  
 7   Promoção                      2870 non-null   object 
 8   Serviço_Telefônico            6418 non-null   object 
 9   Múltiplas_Linhas              5796 non-null   object 
 10  Serviço_de_Internet           6418 non-null   object 
 11  Tipo_de_Internet              5028 non-null   object 
 12  Segurança_Online              5028 non-null   object 
 13  Bac

In [8]:
# prompt: Ver quantos NaN tem em cada coluna de ' data'

data.isnull().sum()


,0
Customer_ID,0
Gênero,0
Idade,0
Casado,0
Estado,0
Número_de_Indicações,0
Tempo_de_Contrato_Meses,0
Promoção,3548
Serviço_Telefônico,0
Múltiplas_Linhas,622


In [9]:
fill_values = {
    'Promoção': 'Nenhuma',
    'Múltiplas_Linhas': 'Não',
    'Tipo_de_Internet': 'Nenhum',
    'Segurança_Online': 'Não',
    'Backup_Online': 'Não',
    'Proteção_de_Dispositivo': 'Não',
    'Suporte_Premium': 'Não',
    'TV_Streaming': 'Não',
    'Filmes_Streaming': 'Não',
    'Música_Streaming': 'Não',
    'Dados_Ilimitados': 'Não',
    'Categoria_de_Churn': 'Outros',
    'Motivo_de_Churn': 'Outros'
}

data.fillna(fill_values, inplace=True)

In [ ]:
# prompt: Saber quantos 1 e quantos 0 tem na coluna df[Status_doChurn]

# Assuming 'df' is your DataFrame and 'Status_doChurn' is the column name.
# Count the occurrences of 0 and 1 in the 'Status_doChurn' column
count_0 = data['Status_do_Churn'].value_counts().get(0, 0)
count_1 = data['Status_do_Churn'].value_counts().get(1, 0)

print(f"Number of 0s in 'Status_doChurn': {count_0}")
print(f"Number of 1s in 'Status_doChurn': {count_1}")


Number of 0s in 'Status_doChurn': 4686
Number of 1s in 'Status_doChurn': 1732


In [ ]:
# prompt: QUERO SABER OS TIPOS DE DADOS DE TODAS AS COLUNAS DE ' data'

data.dtypes


,0
Customer_ID,object
Gênero,object
Idade,int64
Casado,object
Estado,object
Número_de_Indicações,int64
Tempo_de_Contrato_Meses,int64
Promoção,object
Serviço_Telefônico,object
Múltiplas_Linhas,object


In [10]:
data1 = data.copy()

In [11]:
data1

,Customer_ID,Gênero,Idade,Casado,Estado,Número_de_Indicações,Tempo_de_Contrato_Meses,Promoção,Serviço_Telefônico,Múltiplas_Linhas,...,Cobranças_Extras_de_Dados,Cobranças_de_Ligações_Longas,Receita_Total,Status_do_Cliente,Categoria_de_Churn,Motivo_de_Churn,Status_do_Churn,Faixa_Carga_Mensal,Faixa_Idade,Faixa_Tempo_Contrato
0,19877-DEL,Masculino,35,Não,Delhi,7,27,Nenhuma,Sim,Não,...,0,381.51,974.81,Permaneceu,Outros,Outros,0,50-100,20-35,>= 24 Meses
1,58353-MAH,Feminino,45,Sim,Maharashtra,14,13,Nenhuma,Sim,Sim,...,10,96.21,610.28,Permaneceu,Outros,Outros,0,< 20,36-50,12-18 Meses
2,25063-WES,Masculino,51,Não,West Bengal,4,35,Promoção 5,Sim,Não,...,0,134.60,415.45,Churned,Concorrente,Concorrente ofereceu dispositivos melhores,1,50-100,> 50,>= 24 Meses
3,59787-KAR,Masculino,79,Não,Karnataka,3,21,Promoção 4,Sim,Não,...,0,361.66,1599.51,Churned,Insatisfação,Insatisfação com o produto,1,50-100,> 50,18-24 Meses
4,28544-TAM,Feminino,80,Não,Tamil Nadu,3,8,Nenhuma,Sim,Não,...,0,22.14,289.54,Churned,Insatisfação,Problemas com rede,1,50-100,> 50,6-12 Meses
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6413,43397-UTT,Feminino,41,Não,Uttar Pradesh,6,33,Nenhuma,Sim,Sim,...,0,37.96,934.71,Churned,Concorrente,Concorrente fez oferta melhor,1,50-100,36-50,>= 24 Meses
6414,22240-HAR,Masculino,51,Sim,Haryana,0,19,Promoção 1,Não,Não,...,0,0.00,3139.80,Permaneceu,Outros,Outros,0,20-50,> 50,18-24 Meses
6415,78682-KER,Feminino,65,Sim,Kerala,14,25,Nenhuma,Sim,Sim,...,0,599.64,3469.49,Churned,Insatisfação,Insatisfação com o produto,1,50-100,> 50,>= 24 Meses
6416,17816-ODI,Masculino,23,Não,Odisha,2,10,Nenhuma,Sim,Sim,...,0,2107.52,6588.22,Permaneceu,Outros,Outros,0,50-100,20-35,6-12 Meses


In [ ]:
data = data1

In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE

# Carregar os dados

# Remover colunas problemáticas
colunas_removidas = [
    'Customer_ID', 'Status_do_Cliente', 'Motivo_de_Churn',
    'Serviço_Telefônico', 'Total_Reembolsos', 'TV_Streaming',
    'Filmes_Streaming', 'Dados_Ilimitados', 'Proteção_de_Dispositivo',
    'Música_Streaming', 'Backup_Online', 'Múltiplas_Linhas',
    'Categoria_de_Churn'
]
data = data.drop(colunas_removidas, axis=1)

# Definir variáveis e target
colunas_categoricas = [
    'Gênero', 'Casado', 'Estado', 'Promoção', 'Serviço_de_Internet',
    'Tipo_de_Internet', 'Segurança_Online', 'Suporte_Premium', 'Contrato',
    'Fatura_Digital', 'Método_de_Pagamento', 'Faixa_Carga_Mensal',
    'Faixa_Idade', 'Faixa_Tempo_Contrato'
]

X = data.drop('Status_do_Churn', axis=1)
y = data['Status_do_Churn']

# Pré-processamento
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), colunas_categoricas)
    ],
    remainder='passthrough'
)

X_processed = preprocessor.fit_transform(X)

# Divisão dos dados
X_train, X_test, y_train, y_test = train_test_split(
    X_processed,
    y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

# Balanceamento com SMOTE
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_train, y_train)

# Otimização de hiperparâmetros
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'class_weight': ['balanced', {0:1, 1:3}]
}

rf_model = RandomForestClassifier(random_state=42)
search = RandomizedSearchCV(
    rf_model,
    param_grid,
    n_iter=20,
    scoring='recall',
    cv=5,
    n_jobs=-1,
    random_state=42
)

search.fit(X_res, y_res)
best_rf = search.best_estimator_

# Ajuste de threshold
probs = best_rf.predict_proba(X_test)[:,1]
precisions, recalls, thresholds = precision_recall_curve(y_test, probs)

try:
    optimal_threshold = thresholds[(recalls[:-1] > 0.7) & (precisions[:-1] > 0.6)][-1]
except IndexError:
    optimal_threshold = 0.5

y_pred_adj = (probs >= optimal_threshold).astype(int)

# Avaliação final
print("\n" + "="*50)
print("Melhor Modelo Random Forest")
print("="*50)
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred_adj))
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred_adj))

# Feature Importance
feature_names = preprocessor.get_feature_names_out()
importances = best_rf.feature_importances_

print("\n" + "="*50)
print("Top 10 Variáveis Mais Importantes")
print("="*50)
for idx in importances.argsort()[-10:][::-1]:
    print(f"{feature_names[idx]}: {importances[idx]:.4f}")


Melhor Modelo Random Forest
Matriz de Confusão:
[[1188  218]
 [ 155  365]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.88      0.84      0.86      1406
           1       0.63      0.70      0.66       520

    accuracy                           0.81      1926
   macro avg       0.76      0.77      0.76      1926
weighted avg       0.81      0.81      0.81      1926


Top 10 Variáveis Mais Importantes
cat__Contrato_Mensal: 0.1798
cat__Contrato_Dois Anos: 0.0926
cat__Método_de_Pagamento_Cartão de Crédito: 0.0542
remainder__Receita_Total: 0.0440
remainder__Total_Cobrado: 0.0436
cat__Contrato_Um Ano: 0.0422
remainder__Cobrança_Mensal: 0.0348
remainder__Cobranças_de_Ligações_Longas: 0.0336
cat__Tipo_de_Internet_Fibra Óptica: 0.0308
cat__Segurança_Online_Não: 0.0263


In [ ]:
# Após o treinamento (código anterior), adicionar:

import joblib

# Salvar o modelo e pré-processador
joblib.dump(best_rf, 'modelo_churn.pkl')
joblib.dump(preprocessor, 'preprocessor.pkl')
joblib.dump(optimal_threshold, 'threshold.pkl')

['threshold.pkl']

In [ ]:
import pandas as pd
import joblib

# Carregar artefatos salvos
modelo = joblib.load('modelo_churn.pkl')
preprocessor = joblib.load('preprocessor.pkl')
threshold = joblib.load('threshold.pkl')

# Carregar novos dados
novos_dados = data1

# Filtrar apenas não-churn
clientes_ativos = novos_dados[novos_dados['Status_do_Churn'] == 0].copy()

# Preservar IDs
customer_ids = clientes_ativos['Customer_ID']

# Aplicar EXATAMENTE o mesmo pré-processamento do treino
X_novos = preprocessor.transform(clientes_ativos.drop(colunas_removidas, axis=1))

# Fazer previsões
probabilidades = modelo.predict_proba(X_novos)[:, 1]
previsoes = (probabilidades >= threshold).astype(int)

# Criar tabela final
tabela_churn = pd.DataFrame({
    'Customer_ID': customer_ids,
    'Churn_Predict': previsoes,
    'Probabilidade': probabilidades
})

# Filtrar apenas os previstos como churn
churn_risco = tabela_churn[tabela_churn['Churn_Predict'] == 1]

print(f"Clientes em risco: {len(churn_risco)}")
churn_risco.to_csv('clientes_risco_churn.csv', index=False)

Clientes em risco: 577


In [13]:
novos_dados = data1

# Filtrar apenas não-churn
clientes_ativos = novos_dados[novos_dados['Status_do_Churn'] == 0].copy()


In [14]:
clientes_ativos1 = clientes_ativos.copy()

In [ ]:
data1

,Customer_ID,Gênero,Idade,Casado,Estado,Número_de_Indicações,Tempo_de_Contrato_Meses,Promoção,Serviço_Telefônico,Múltiplas_Linhas,...,Cobranças_Extras_de_Dados,Cobranças_de_Ligações_Longas,Receita_Total,Status_do_Cliente,Categoria_de_Churn,Motivo_de_Churn,Status_do_Churn,Faixa_Carga_Mensal,Faixa_Idade,Faixa_Tempo_Contrato
0,19877-DEL,Masculino,35,Não,Delhi,7,27,Nenhuma,Sim,Não,...,0,381.51,974.81,Permaneceu,Outros,Outros,0,50-100,20-35,>= 24 Meses
1,58353-MAH,Feminino,45,Sim,Maharashtra,14,13,Nenhuma,Sim,Sim,...,10,96.21,610.28,Permaneceu,Outros,Outros,0,< 20,36-50,12-18 Meses
2,25063-WES,Masculino,51,Não,West Bengal,4,35,Promoção 5,Sim,Não,...,0,134.60,415.45,Churned,Concorrente,Concorrente ofereceu dispositivos melhores,1,50-100,> 50,>= 24 Meses
3,59787-KAR,Masculino,79,Não,Karnataka,3,21,Promoção 4,Sim,Não,...,0,361.66,1599.51,Churned,Insatisfação,Insatisfação com o produto,1,50-100,> 50,18-24 Meses
4,28544-TAM,Feminino,80,Não,Tamil Nadu,3,8,Nenhuma,Sim,Não,...,0,22.14,289.54,Churned,Insatisfação,Problemas com rede,1,50-100,> 50,6-12 Meses
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6413,43397-UTT,Feminino,41,Não,Uttar Pradesh,6,33,Nenhuma,Sim,Sim,...,0,37.96,934.71,Churned,Concorrente,Concorrente fez oferta melhor,1,50-100,36-50,>= 24 Meses
6414,22240-HAR,Masculino,51,Sim,Haryana,0,19,Promoção 1,Não,Não,...,0,0.00,3139.80,Permaneceu,Outros,Outros,0,20-50,> 50,18-24 Meses
6415,78682-KER,Feminino,65,Sim,Kerala,14,25,Nenhuma,Sim,Sim,...,0,599.64,3469.49,Churned,Insatisfação,Insatisfação com o produto,1,50-100,> 50,>= 24 Meses
6416,17816-ODI,Masculino,23,Não,Odisha,2,10,Nenhuma,Sim,Sim,...,0,2107.52,6588.22,Permaneceu,Outros,Outros,0,50-100,20-35,6-12 Meses


In [ ]:
'Serviço_Telefônico','Total_Reembolso','TV_Streaming','Filmes_Straming','Dados_Ilimitados','Proteção_de_Dispositivo','Musica_Straming','Backup_Online','Multiplas_Linhas'

In [12]:
caminho = '/content/_Predictions.csv'
datapredicted = pd.read_csv(caminho)

In [15]:
clientes_ativos.columns

Index(['Customer_ID', 'Gênero', 'Idade', 'Casado', 'Estado',
       'Número_de_Indicações', 'Tempo_de_Contrato_Meses', 'Promoção',
       'Serviço_Telefônico', 'Múltiplas_Linhas', 'Serviço_de_Internet',
       'Tipo_de_Internet', 'Segurança_Online', 'Backup_Online',
       'Proteção_de_Dispositivo', 'Suporte_Premium', 'TV_Streaming',
       'Filmes_Streaming', 'Música_Streaming', 'Dados_Ilimitados', 'Contrato',
       'Fatura_Digital', 'Método_de_Pagamento', 'Cobrança_Mensal',
       'Total_Cobrado', 'Total_Reembolsos', 'Cobranças_Extras_de_Dados',
       'Cobranças_de_Ligações_Longas', 'Receita_Total', 'Status_do_Cliente',
       'Categoria_de_Churn', 'Motivo_de_Churn', 'Status_do_Churn',
       'Faixa_Carga_Mensal', 'Faixa_Idade', 'Faixa_Tempo_Contrato'],
      dtype='object')

In [16]:
datapredicted

,Customer_ID,Gênero,Idade,Casado,Estado,Número_de_Indicações,Tempo_de_Contrato_Meses,Promoção,Serviço_Telefônico,Múltiplas_Linhas,...,Cobranças_de_Ligações_Longas,Receita_Total,Status_do_Cliente,Categoria_de_Churn,Motivo_de_Churn,Status_do_Churn,Faixa_Carga_Mensal,Faixa_Idade,Faixa_Tempo_Contrato,Customer_Status_Predicted
0,25063-WES,Masculino,51,Não,West Bengal,4,35,Promoção 5,Sim,Não,...,134.60,415.45,Churned,Concorrente,Concorrente ofereceu dispositivos melhores,1,50-100,> 50,>= 24 Meses,1
1,59787-KAR,Masculino,79,Não,Karnataka,3,21,Promoção 4,Sim,Não,...,361.66,1599.51,Churned,Insatisfação,Insatisfação com o produto,1,50-100,> 50,18-24 Meses,1
2,28544-TAM,Feminino,80,Não,Tamil Nadu,3,8,Nenhuma,Sim,Não,...,22.14,289.54,Churned,Insatisfação,Problemas com rede,1,50-100,> 50,6-12 Meses,1
3,73360-AND,Feminino,74,Não,Andhra Pradesh,0,30,Nenhuma,Sim,Não,...,480.15,3271.65,Churned,Insatisfação,Pouca variedade de serviços,1,50-100,> 50,>= 24 Meses,1
4,36756-MAD,Masculino,64,Sim,Madhya Pradesh,15,33,Nenhuma,Não,Não,...,0.00,25.10,Churned,Concorrente,Concorrente fez oferta melhor,1,20-50,> 50,>= 24 Meses,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1625,82479-TEL,Feminino,72,Sim,Telangana,8,26,Nenhuma,Sim,Sim,...,112.44,421.14,Churned,Concorrente,Concorrente ofereceu mais dados,1,50-100,> 50,>= 24 Meses,1
1626,81946-TAM,Feminino,55,Sim,Tamil Nadu,7,3,Nenhuma,Sim,Não,...,156.45,1286.45,Permaneceu,Outros,Outros,0,50-100,> 50,< 6 Meses,1
1627,56232-UTT,Feminino,79,Sim,Uttar Pradesh,3,35,Nenhuma,Sim,Sim,...,785.40,2810.50,Permaneceu,Outros,Outros,0,50-100,> 50,>= 24 Meses,1
1628,68167-MAH,Feminino,56,Não,Maharashtra,15,25,Promoção 5,Sim,Sim,...,30.52,115.52,Churned,Outro,Não sabe/Não informado,1,50-100,> 50,>= 24 Meses,1


In [ ]:
import pandas as pd

# Carregar os dados
clientes_risco_churn = datapredicted

# Combinar as tabelas via Customer_ID (LEFT JOIN)
clientes_predictos = pd.merge(
    clientes_risco_churn,
    clientes_ativos1,
    on='Customer_ID',
    how='left'
)

# Salvar o resultado
clientes_predictos.to_csv('clientes_predictos.csv', index=False)

In [17]:
caminho2 = '/content/clientes_predictos.csv'

clientespred = pd.read_csv(caminho2)

In [18]:
clientespred.sort_values(by='Probabilidade', ascending=False)

,Customer_ID,Churn_Predict,Probabilidade,Gênero,Idade,Casado,Estado,Número_de_Indicações,Tempo_de_Contrato_Meses,Promoção,...,Cobranças_Extras_de_Dados,Cobranças_de_Ligações_Longas,Receita_Total,Status_do_Cliente,Categoria_de_Churn,Motivo_de_Churn,Status_do_Churn,Faixa_Carga_Mensal,Faixa_Idade,Faixa_Tempo_Contrato
400,74779-UTT,1,0.959376,Masculino,67,Sim,Uttar Pradesh,5,34,Promoção 5,...,0,289.92,787.52,Permaneceu,Outros,Outros,0,50-100,> 50,>= 24 Meses
460,51751-KER,1,0.958717,Feminino,62,Não,Kerala,9,25,Nenhuma,...,0,67.50,191.30,Novo Cliente,Outros,Outros,0,50-100,> 50,>= 24 Meses
211,14658-RAJ,1,0.953464,Feminino,65,Não,Rajasthan,12,24,Nenhuma,...,0,384.37,1713.52,Permaneceu,Outros,Outros,0,50-100,> 50,>= 24 Meses
104,78253-GUJ,1,0.953436,Masculino,63,Não,Gujarat,13,30,Nenhuma,...,0,15.20,100.20,Novo Cliente,Outros,Outros,0,50-100,> 50,>= 24 Meses
404,66413-TAM,1,0.952780,Feminino,53,Não,Tamil Nadu,12,4,Nenhuma,...,0,80.58,312.93,Novo Cliente,Outros,Outros,0,50-100,> 50,< 6 Meses
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,83994-DEL,1,0.650179,Masculino,48,Sim,Delhi,8,24,Nenhuma,...,0,16.17,253.82,Novo Cliente,Outros,Outros,0,50-100,36-50,>= 24 Meses
351,55004-TEL,1,0.650018,Masculino,27,Sim,Telangana,6,25,Promoção 2,...,0,0.00,1453.10,Permaneceu,Outros,Outros,0,20-50,20-35,>= 24 Meses
283,19020-KER,1,0.649744,Feminino,59,Não,Kerala,1,20,Nenhuma,...,0,67.70,302.90,Permaneceu,Outros,Outros,0,20-50,> 50,18-24 Meses
510,73015-KER,1,0.649002,Masculino,28,Não,Kerala,3,27,Nenhuma,...,0,75.05,492.22,Permaneceu,Outros,Outros,0,50-100,20-35,>= 24 Meses


In [ ]:
clientespred

,Customer_ID,Churn_Predict,Probabilidade,Gênero,Idade,Casado,Estado,Número_de_Indicações,Tempo_de_Contrato_Meses,Promoção,...,Cobranças_Extras_de_Dados,Cobranças_de_Ligações_Longas,Receita_Total,Status_do_Cliente,Categoria_de_Churn,Motivo_de_Churn,Status_do_Churn,Faixa_Carga_Mensal,Faixa_Idade,Faixa_Tempo_Contrato
0,13717-UTT,1,0.944016,Masculino,56,Sim,Uttar Pradesh,15,26,Nenhuma,...,0,93.25,444.75,Permaneceu,Outros,Outros,0,50-100,> 50,>= 24 Meses
1,86818-MAH,1,0.841330,Feminino,29,Sim,Maharashtra,8,22,Promoção 4,...,0,802.93,2652.88,Permaneceu,Outros,Outros,0,50-100,20-35,18-24 Meses
2,76566-UTT,1,0.902227,Masculino,61,Não,Uttar Pradesh,1,20,Promoção 3,...,0,682.02,2611.02,Permaneceu,Outros,Outros,0,50-100,> 50,18-24 Meses
3,28648-DEL,1,0.740841,Masculino,54,Sim,Delhi,3,14,Nenhuma,...,20,59.45,2687.05,Permaneceu,Outros,Outros,0,50-100,> 50,12-18 Meses
4,33514-UTT,1,0.672581,Feminino,28,Sim,Uttar Pradesh,2,3,Promoção 5,...,0,255.96,570.51,Permaneceu,Outros,Outros,0,50-100,20-35,< 6 Meses
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,32117-TAM,1,0.906128,Feminino,25,Sim,Tamil Nadu,7,26,Nenhuma,...,60,422.40,2304.35,Permaneceu,Outros,Outros,0,50-100,20-35,>= 24 Meses
573,46848-WES,1,0.656386,Feminino,22,Sim,West Bengal,14,10,Nenhuma,...,0,42.90,778.40,Permaneceu,Outros,Outros,0,50-100,20-35,6-12 Meses
574,81946-TAM,1,0.941953,Feminino,55,Sim,Tamil Nadu,7,3,Nenhuma,...,0,156.45,1286.45,Permaneceu,Outros,Outros,0,50-100,> 50,< 6 Meses
575,56232-UTT,1,0.938153,Feminino,79,Sim,Uttar Pradesh,3,35,Nenhuma,...,0,785.40,2810.50,Permaneceu,Outros,Outros,0,50-100,> 50,>= 24 Meses


In [19]:
# Cálculo de métricas gerais
total_clientes_previsto = len(clientespred)

# Churn por gênero
contagem_pred_demografico = clientespred[clientespred['Churn_Predict'] == 1].groupby('Gênero').size().reset_index(name='Total de previstos')
print("Churn por Gênero:\n", contagem_pred_demografico)

# Churn por método de pagamento
total_pred_por_tipo_pagamento = clientespred.groupby('Método_de_Pagamento')['Churn_Predict'].sum().reset_index()
total_pred_por_tipo_pagamento = total_pred_por_tipo_pagamento.sort_values(by='Churn_Predict', ascending=False)
print("\nPred por Método de Pagamento:\n", total_pred_por_tipo_pagamento)

# Churn por tipo de contrato
total_pred_por_contrato = clientespred.groupby('Contrato')['Churn_Predict'].sum().reset_index()
total_pred_por_contrato = total_pred_por_contrato.sort_values(by='Churn_Predict', ascending=False)
print("\nPred por Tipo de Contrato:\n", total_pred_por_contrato)

# Churn por estado (top 5)
total_pred_estados = clientespred.groupby('Estado')['Churn_Predict'].sum().reset_index()
total_pred_estados_top5 = total_pred_estados.sort_values(by='Churn_Predict', ascending=False).head(5)
print("\nTop 5 Estados com Mais previsões:\n", total_pred_estados_top5)

# Churn por categoria e tipo de internet
total_pred_categoria = clientespred.groupby('Categoria_de_Churn')['Churn_Predict'].sum().sort_values(ascending=False)
total_pred_tipo_internet = clientespred.groupby('Tipo_de_Internet')['Churn_Predict'].sum().sort_values(ascending=False)

print("\nPred por Categoria:\n", total_pred_categoria)

print("\nPred por Tipo de Internet:\n", total_pred_tipo_internet)

# Distribuição por faixa etária (Eixo X: Faixa, Eixo Y: Total)
# =====================================================
ordem_idade = ['< 20', '20-35', '36-50', '> 50']
total_cliente_grupo_idade_pred = clientespred.groupby('Faixa_Idade').size().reset_index(name='Total_Clientes')
total_cliente_grupo_idade_pred['Faixa_Idade'] = pd.Categorical(total_cliente_grupo_idade_pred['Faixa_Idade'], categories=ordem_idade, ordered=True)
total_cliente_grupo_idade_pred = total_cliente_grupo_idade_pred.sort_values('Faixa_Idade')
print("\nClientes por Faixa Etária:\n", total_cliente_grupo_idade_pred)

# =====================================================
# Distribuição por tempo de contrato (Eixo X: Faixa, Eixo Y: Total)
# =====================================================
ordem_tempo = ['< 6 Meses', '6-12 Meses', '12-18 Meses', '18-24 Meses', '>= 24 Meses']
total_clientes_tempo_contratado_pred = clientespred.groupby('Faixa_Tempo_Contrato').size().reset_index(name='Total_Clientes')
total_clientes_tempo_contratado_pred['Faixa_Tempo_Contrato'] = pd.Categorical(total_clientes_tempo_contratado_pred['Faixa_Tempo_Contrato'], categories=ordem_tempo, ordered=True)
total_clientes_tempo_contratado_pred = total_clientes_tempo_contratado_pred.sort_values('Faixa_Tempo_Contrato')
print("\nClientes por Tempo de Contrato:\n", total_clientes_tempo_contratado_pred)

Churn por Gênero:
       Gênero  Total de previstos
0   Feminino                 379
1  Masculino                 198

Pred por Método de Pagamento:
   Método_de_Pagamento  Churn_Predict
2     Débito em Conta            462
0   Cartão de Crédito             86
1  Cheque por Correio             29

Pred por Tipo de Contrato:
   Contrato  Churn_Predict
0   Mensal            577

Top 5 Estados com Mais previsões:
            Estado  Churn_Predict
19  Uttar Pradesh             55
17     Tamil Nadu             51
12    Maharashtra             49
9       Karnataka             42
6         Haryana             41

Pred por Categoria:
 Categoria_de_Churn
Outros    577
Name: Churn_Predict, dtype: int64

Pred por Tipo de Internet:
 Tipo_de_Internet
Fibra Óptica    420
DSL              97
Cabo             56
Nenhum            4
Name: Churn_Predict, dtype: int64

Clientes por Faixa Etária:
   Faixa_Idade  Total_Clientes
2        < 20              16
0       20-35             156
1       36-50      

In [27]:
import dash_bootstrap_components as dbc
from dash import dcc, html, Dash, Input, Output, dash_table
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# CONSTANTES DE CORES
COR_PRIMARIA = "#8e44ad"
COR_SECUNDARIA = "#e74c3c"
COR_SUCESSO = "#27ae60"
COR_ALERTA = "#c0392b"
COR_AVISO = "#f39c12"
FUNDO_ESCURO = "#1a1a1a"
TEXTO_CLARO = "#ffffff"
SLOT_CLARO = "#ffffff"

todos_estados = pd.Series({
    'Uttar Pradesh': 629, 'Tamil Nadu': 600, 'Maharashtra': 504,
    'Karnataka': 470, 'Haryana': 398, 'Andhra Pradesh': 395,
    'West Bengal': 368, 'Punjab': 342, 'Bihar': 336, 'Gujarat': 335,
    'Jammu & Kashmir': 320, 'Madhya Pradesh': 288, 'Telangana': 281,
    'Rajasthan': 259, 'Kerala': 200, 'Odisha': 152, 'Assam': 139,
    'Delhi': 127, 'Jharkhand': 113, 'Uttarakhand': 62,
    'Chhattisgarh': 59, 'Puducherry': 41
})

# ================================
# FUNÇÕES AUXILIARES
# ================================
def criar_card(valor, titulo, cor_fundo, icone, card_id):
    return dbc.Card(
        dbc.CardBody([
            html.Div(className="d-flex align-items-center", children=[
                html.Div(className="icon-container me-3", children=[
                    html.I(className=icone, style={"fontSize": "2.5rem", "color": cor_fundo})
                ]),
                html.Div([
                    html.H2(valor, className="card-value mb-0", style={'color': cor_fundo}),
                    html.P(titulo, className="card-title mb-0", style={'color': '#555555'})
                ])
            ])
        ]),
        className="shadow-lg metric-card",
        style={
            'borderRadius': '15px',
            'backgroundColor': SLOT_CLARO,
            'transition': 'transform 0.2s',
            'margin': '0.5rem',
            'border': '1px solid #e0e0e0'
        },
        id=card_id
    )

def criar_grafico_combinado(total_df, taxa_series, titulo, eixo_x):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(
        go.Bar(
            x=total_df.iloc[:, 0],
            y=total_df['Total_Clientes'],
            name='Total de Clientes',
            marker_color=COR_PRIMARIA,
            opacity=0.7,
            text=total_df['Total_Clientes'],
            textposition='outside',
            texttemplate='%{text:,}',
            textfont=dict(color=TEXTO_CLARO)
        ),
        secondary_y=False
    )
    fig.add_trace(
        go.Scatter(
            x=taxa_series.index,
            y=taxa_series.values,
            name='Taxa de Churn',
            line=dict(color='red', width=2),
            mode='lines+markers+text',
            marker=dict(size=6, symbol='diamond'),
            text=[f'{val:.1f}%' for val in taxa_series.values],
            textposition='top center',
            textfont=dict(color=TEXTO_CLARO)
        ),
        secondary_y=True
    )
    max_churn = max(taxa_series.values)
    max_tick = 10 * ((int(max_churn) // 10 + 1))
    fig.update_layout(
        title_text=titulo,
        template='plotly_dark',
        height=320,
        margin=dict(l=15, r=15, t=60, b=20),
        legend=dict(orientation="h", yanchor="bottom", y=1.02),
        xaxis_title=eixo_x,
        xaxis=dict(showgrid=False, color=TEXTO_CLARO),
        yaxis=dict(showgrid=False, color=TEXTO_CLARO),
        title_font=dict(size=18, family='Arial', color=TEXTO_CLARO),
        title_x=0.5,
        paper_bgcolor=FUNDO_ESCURO,
        plot_bgcolor=FUNDO_ESCURO
    )
    fig.update_yaxes(title_text="Clientes", secondary_y=False, showgrid=False)
    fig.update_yaxes(title_text="Churn (%)", secondary_y=True,
                   range=[0, max_tick], showgrid=False)
    return fig

def criar_grafico_horizontal(data_series, titulo, label_y, label_x, formato_percent=False):
    if formato_percent:
        text_template = '%{text:.1f}%'
        tick_format = '.1f%'
    else:
        text_template = '%{text:,}'
        tick_format = ','
    fig = px.bar(
        y=[str(i) for i in data_series.index],
        x=data_series.values,
        orientation='h',
        title=titulo,
        labels={'y': label_y, 'x': label_x},
        color_discrete_sequence=[COR_PRIMARIA]
    )
    fig.update_traces(
        text=data_series.values,
        textposition='outside',
        texttemplate=text_template,
        textfont=dict(color=TEXTO_CLARO)
    )
    fig.update_layout(
        template='plotly_dark',
        height=280,
        margin=dict(l=15, r=15, t=60, b=20),
        xaxis=dict(
            showgrid=False,
            tickformat=tick_format,
            range=[0, data_series.max() * 1.15],
            color=TEXTO_CLARO
        ),
        yaxis=dict(showgrid=False, color=TEXTO_CLARO),
        title_font=dict(size=16, family='Arial', color=TEXTO_CLARO),
        title_x=0.5,
        paper_bgcolor=FUNDO_ESCURO,
        plot_bgcolor=FUNDO_ESCURO
    )
    return fig

def criar_mapa_churn():
    df_map = todos_estados.reset_index()
    df_map.columns = ['Estado', 'Churn']
    df_map['Estado'] = df_map['Estado'].replace({
        'Jammu & Kashmir': 'Jammu and Kashmir',
        'Andhra Pradesh': 'Andhra Pradesh',
        'Uttar Pradesh': 'Uttar Pradesh'
    })
    geojson_url = "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
    fig = px.choropleth_mapbox(
        df_map,
        geojson=geojson_url,
        locations='Estado',
        featureidkey='properties.ST_NM',
        color='Churn',
        color_continuous_scale='Viridis',
        range_color=(0, todos_estados.max()),
        mapbox_style="carto-darkmatter",
        zoom=3.5,
        center={"lat": 22.9734, "lon": 78.6569},
        opacity=0.8,
        hover_data={'Estado': True, 'Churn': ':,.0f'},
        labels={'Churn': 'Casos de Churn'}
    )
    fig.update_layout(
        title={
            'text': "<b>Distribuição de Churn por Estado</b>",
            'font': {'size': 24, 'color': TEXTO_CLARO, 'family': 'Arial Black'},
            'x': 0.5,
            'y': 0.95,
            'xanchor': 'center',
            'yanchor': 'top',
            'pad': {'b': 20}
        },
        margin={"r":20,"t":80,"l":20,"b":20},
        coloraxis_colorbar=dict(
            title='Casos de Churn',
            thickness=15,
            len=0.6,
            title_font=dict(color=TEXTO_CLARO, size=12),
            tickfont=dict(color=TEXTO_CLARO, size=10),
            orientation='v',
            yanchor='middle',
            y=0.5,
            x=0.93
        ),
        paper_bgcolor=FUNDO_ESCURO,
        plot_bgcolor=FUNDO_ESCURO
    )
    fig.update_geos(
        visible=False,
        bgcolor='rgba(0,0,0,0)',
        lakecolor='#2a2a2a',
        landcolor='#1a1a1a',
        showcountries=True,
        countrycolor='#444444',
        countrywidth=0.5
    )
    fig.update_traces(
        hovertemplate="<b>%{customdata[0]}</b><br>" +
                      "Churn: %{customdata[1]}<extra></extra>",
        marker=dict(line=dict(width=0.5, color='#333333'))
    )
    mapbox_layers=[
            {
                "sourcetype": "geojson",
                "source": geojson_url,
                "type": "symbol",
                "text-field": ["get", "ST_NM"],
                "text-size": 12,
                "text-color": "white",
                "text-halo-color": "#2a2a2a",
                "text-halo-width": 1
            }
        ]
    return fig

# ================================
# COMPONENTES PRINCIPAIS
# ================================
cabecalho_e_metricas = [
    dbc.Row(
        dbc.Col(
            html.Div([
                html.Div([
                    html.I(className="fas fa-chart-line me-2",
                          style={"color": COR_PRIMARIA, "fontSize": "2.5rem"}),
                    html.H1("Dashboard de Churn",
                           className="display-4 mb-0",
                           style={'color': TEXTO_CLARO})
                ], className="d-flex justify-content-center align-items-center"),
                html.P("Análise de Retenção de Clientes",
                      className="lead",
                      style={'color': COR_PRIMARIA})
            ], className="text-center py-4"),
            width=12
        )
    ),
    dbc.Row([
        dbc.Col(criar_card(
            f"{total_clientes:,}",
            "Total Clientes",
            COR_PRIMARIA,
            "fas fa-users",
            {'type': 'metric-card', 'index': 0}
        ), width=3),
        dbc.Col(criar_card(
            f"{novos_clientes:,}",
            "Novos Clientes",
            COR_SUCESSO,
            "fas fa-user-plus",
            {'type': 'metric-card', 'index': 1}
        ), width=3),
        dbc.Col(criar_card(
            f"{total_churn:,}",
            "Clientes Perdidos",
            COR_ALERTA,
            "fas fa-user-minus",
            {'type': 'metric-card', 'index': 2}
        ), width=3),
        dbc.Col(criar_card(
            f"{taxa_churn_geral:.1f}%",
            "Taxa de Churn",
            COR_AVISO,
            "fas fa-percentage",
            {'type': 'metric-card', 'index': 3}
        ), width=3)
    ], className="g-4 mb-4")
]

# ================================
# CRIAÇÃO DOS GRÁFICOS
# ================================
fig_faixa_etaria = criar_grafico_combinado(
    total_cliente_grupo_idade,
    taxa_churn_grupo_idade,
    "Churn por Faixa Etária",
    "Faixa Etária"
)

fig_tempo_contrato = criar_grafico_combinado(
    Total_clientes_por_tempo_contratado,
    taxa_churn_por_tempo_contratado,
    "Churn por Tempo de Contrato",
    "Meses de Contrato"
)

fig_churn_genero = px.pie(
    names=contagem_churn_demografico['Gênero'],
    values=contagem_churn_demografico['Total de Churn'],
    title='Churn por Gênero',
    color_discrete_sequence=[COR_PRIMARIA, COR_SECUNDARIA]
)

legenda_genero = html.Div([
    html.Table(
        className="table",
        children=[
            html.Thead(html.Tr([
                html.Th("Cor"),
                html.Th("Gênero"),
                html.Th("Clientes")
            ])),
            html.Tbody([
                html.Tr([
                    html.Td(html.Div(style={'width': '20px', 'height': '20px', 'backgroundColor': COR_PRIMARIA})),
                    html.Td("Feminino"),
                    html.Td(f"{contagem_churn_demografico[contagem_churn_demografico['Gênero'] == 'Feminino']['Total de Churn'].values[0]:,}")
                ]),
                html.Tr([
                    html.Td(html.Div(style={'width': '20px', 'height': '20px', 'backgroundColor': COR_SECUNDARIA})),
                    html.Td("Masculino"),
                    html.Td(f"{contagem_churn_demografico[contagem_churn_demografico['Gênero'] == 'Masculino']['Total de Churn'].values[0]:,}")
                ])
            ])
        ],
        style={
            'marginTop': '20px',
            'backgroundColor': FUNDO_ESCURO,
            'color': TEXTO_CLARO,
            'width': '100%'
        }
    )
])

fig_churn_genero.update_traces(
    textinfo='label+percent+value',
    texttemplate='%{label}<br>%{value} (%{percent})',
    marker=dict(line=dict(color='black', width=1)),
    textfont=dict(color=TEXTO_CLARO, size=14),
    hovertemplate="<b>%{label}</b><br>Quantidade: %{value}<br>Porcentagem: %{percent}<extra></extra>",
    textposition='auto',
    insidetextorientation='horizontal'
)
fig_churn_genero.update_layout(
    template='plotly_dark',
    height=320,
    margin=dict(l=15, r=15, t=60, b=20),
    title_font=dict(size=18, family='Arial', color=TEXTO_CLARO),
    title_x=0.5,
    paper_bgcolor=FUNDO_ESCURO,
    showlegend=False
)

fig_taxa_pagamento = criar_grafico_horizontal(
    taxa_churn_por_tipo_pagamento,
    'Taxa de Churn por Método de Pagamento',
    'Método de Pagamento',
    'Taxa de Churn (%)',
    formato_percent=True
)

fig_churn_contrato = criar_grafico_horizontal(
    taxa_churn_por_contrato,
    'Taxa de Churn por Tipo de Contrato',
    'Tipo de Contrato',
    'Taxa de Churn (%)',
    formato_percent=True
)

fig_estado = criar_grafico_horizontal(
    todos_estados.sort_values(ascending=False).head(5),
    'Taxa de Churn por Estado (Top 5)',
    'Estado',
    'Taxa de Churn (%)',
    formato_percent=False
)

fig_categoria = criar_grafico_horizontal(
    total_churn_categoria,
    'Total de Churn por Categoria',
    'Categoria',
    'Clientes',
    formato_percent=False
)

fig_internet = criar_grafico_horizontal(
    total_churn_tipo_internet,
    'Total de Churn por Tipo de Internet',
    'Tipo de Internet',
    'Clientes',
    formato_percent=False
)

# ================================
# LAYOUT PRINCIPAL COM ABAS
# ================================
app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc.icons.FONT_AWESOME])
app.layout = dbc.Container([
    dbc.Tabs([
        # Aba Principal Existente
        dbc.Tab(
            label="Dashboard Principal",
            children=[
                *cabecalho_e_metricas,
                dbc.Row([
                    dbc.Col(
                        dbc.Card(
                            html.Div([
                                dcc.Graph(figure=fig_churn_genero),
                                legenda_genero
                            ]),
                            style={'backgroundColor': FUNDO_ESCURO, 'border': '1px solid #333'}
                        ), width=4
                    ),
                    dbc.Col(
                        dbc.Card(
                            dcc.Graph(figure=fig_faixa_etaria),
                            style={'backgroundColor': FUNDO_ESCURO, 'border': '1px solid #333'}
                        ), width=4
                    ),
                    dbc.Col(
                        dbc.Card(
                            dcc.Graph(figure=fig_tempo_contrato),
                            style={'backgroundColor': FUNDO_ESCURO, 'border': '1px solid #333'}
                        ), width=4
                    )
                ], className="mb-4"),
                dbc.Row([
                    dbc.Col(
                        dbc.Card(
                            dcc.Graph(figure=fig_taxa_pagamento),
                            style={'backgroundColor': FUNDO_ESCURO, 'border': '1px solid #333'}
                        ), width=3
                    ),
                    dbc.Col(
                        dbc.Card(
                            dcc.Graph(figure=fig_churn_contrato),
                            style={'backgroundColor': FUNDO_ESCURO, 'border': '1px solid #333'}
                        ), width=3
                    ),
                    dbc.Col(
                        dbc.Card(
                            dcc.Graph(figure=fig_estado),
                            style={'backgroundColor': FUNDO_ESCURO, 'border': '1px solid #333'}
                        ), width=3
                    ),
                    dbc.Col(
                        dbc.Card(
                            dcc.Graph(figure=fig_categoria),
                            style={'backgroundColor': FUNDO_ESCURO, 'border': '1px solid #333'}
                        ), width=3
                    ),
                    dbc.Col(
                        dbc.Card(
                            dcc.Graph(figure=fig_internet),
                            style={'backgroundColor': FUNDO_ESCURO, 'border': '1px solid #333'}
                        ), width=3
                    )
                ], className="mb-4 g-2", justify="around")
            ],
            tabClassName="fw-bold"
        ),

        # Aba de Mapa Existente
        dbc.Tab(
            label="Mapa de Churn",
            children=[
                dbc.Row([
                    dbc.Col(
                        dbc.Card(
                            dcc.Graph(
                                id='mapa-churn',
                                figure=criar_mapa_churn(),
                                config={
                                    'displayModeBar': True,
                                    'scrollZoom': True,
                                    'modeBarButtonsToAdd': ['zoomInMapbox', 'zoomOutMapbox']
                                }
                            ),
                            style={
                                'border': f'2px solid {COR_PRIMARIA}',
                                'borderRadius': '15px',
                                'backgroundColor': FUNDO_ESCURO
                            }
                        ), width=12
                    )
                ], className="mt-4")
            ],
            tabClassName="fw-bold"
        ),

        # NOVA ABA DE ANÁLISE PREDITIVA
        dbc.Tab(
            label="Análise Preditiva",
            children=[
                # Cabeçalho
                dbc.Row([
                    dbc.Col(html.H1("Dashboard de Churn - Análise Preditiva",
                                   className='text-center mb-4',
                                   style={'color': COR_PRIMARIA, 'fontWeight': 'bold'}), width=12)
                ]),

                # Seção de Métricas e Tabela
                dbc.Row([
                    dbc.Col([
                        dbc.Card(style={'backgroundColor': FUNDO_ESCURO, 'border': f'2px solid {COR_PRIMARIA}'}, children=[
                            dbc.CardBody([
                                html.H4("Total de Clientes Previstos", className="card-title", style={'color': COR_PRIMARIA}),
                                html.H3(id='total-clientes', children=f"{total_clientes_previsto}",
                                       className="card-text text-center", style={'color': TEXTO_CLARO, 'fontWeight': 'bold'})
                            ])
                        ])
                    ], width=3),

                    dbc.Col([
                        dbc.Card(style={'backgroundColor': FUNDO_ESCURO, 'border': f'2px solid {COR_PRIMARIA}'}, children=[
                            dbc.CardBody([
                                html.H5("Clientes com Risco de Churn", className="card-title", style={'color': COR_PRIMARIA}),
                                dash_table.DataTable(
                                    id='clientes-pred-table',
                                    columns=[
                                        {'name': 'Customer ID', 'id': 'Customer_ID'},
                                        {'name': 'Idade', 'id': 'Idade'},
                                        {'name': 'Estado', 'id': 'Estado'}
                                    ],
                                    data=clientespred.to_dict('records'),
                                    page_size=10,
                                    style_table={
                                        'overflowX': 'auto',
                                        'height': '400px',
                                        'backgroundColor': FUNDO_ESCURO
                                    },
                                    style_cell={
                                        'textAlign': 'left',
                                        'padding': '8px',
                                        'backgroundColor': FUNDO_ESCURO,
                                        'color': TEXTO_CLARO,
                                        'border': f'1px solid {FUNDO_ESCURO}'
                                    },
                                    style_header={
                                        'backgroundColor': FUNDO_ESCURO,
                                        'fontWeight': 'bold',
                                        'color': TEXTO_CLARO,
                                        'borderBottom': f'2px solid {FUNDO_ESCURO}'
                                    },
                                    style_data_conditional=[
                                        {'if': {'row_index': 'odd'},
                                         'backgroundColor': '#373737'}
                                    ]
                                )
                            ])
                        ])
                    ], width=9)
                ], className='mb-4'),

                # Primeira Linha de Gráficos
               dbc.Row([
                    dbc.Col([
                        dbc.Card(
                            style={
                                'backgroundColor': FUNDO_ESCURO,
                                'border': f'2px solid {COR_PRIMARIA}',
                                'height': '100%',
                                'display': 'flex',
                                'flexDirection': 'column'
                            },
                            children=[
                                dbc.CardBody([
                                    html.H5("Churn por Gênero",
                                           className="card-title mb-4",
                                           style={'color': COR_PRIMARIA, 'textAlign': 'center'}),

                                    # Gráfico de Pizza
                                    dcc.Graph(
                                        figure=px.pie(
                                            contagem_pred_demografico,
                                            names='Gênero',
                                            values='Total de previstos',
                                            title='Distribuição por Gênero',
                                            color_discrete_sequence=[COR_PRIMARIA, COR_SECUNDARIA],
                                            template='plotly_dark'
                                        ).update_layout(
                                            paper_bgcolor=FUNDO_ESCURO,
                                            plot_bgcolor=FUNDO_ESCURO,
                                            autosize=False,
                                            width=380,
                                            height=380,
                                            margin=dict(l=20, r=20, t=40, b=20),
                                            title_x=0.5
                                        ),
                                        style={"width": "100%", "height": "300px"}
                                    ),

                                    # Cards de Legendas
                                    html.Div([
                                        dbc.Row([
                                            dbc.Col([
                                                dbc.Card(
                                                    style={'backgroundColor': COR_PRIMARIA, 'borderRadius': '10px'},
                                                    children=[
                                                        dbc.CardBody([
                                                            html.H6("Feminino",
                                                                   style={'color': 'white', 'textAlign': 'center'}),
                                                            html.H4(f"{contagem_pred_demografico[contagem_pred_demografico['Gênero'] == 'Feminino']['Total de previstos'].values[0]:,}",
                                                                   style={'color': 'white', 'textAlign': 'center'})
                                                        ])
                                                    ])
                                            ], width=6),
                                            dbc.Col([
                                                dbc.Card(
                                                    style={'backgroundColor': COR_SECUNDARIA, 'borderRadius': '10px'},
                                                    children=[
                                                        dbc.CardBody([
                                                            html.H6("Masculino",
                                                                   style={'color': 'white', 'textAlign': 'center'}),
                                                            html.H4(f"{contagem_pred_demografico[contagem_pred_demografico['Gênero'] == 'Masculino']['Total de previstos'].values[0]:,}",
                                                                   style={'color': 'white', 'textAlign': 'center'})
                                                        ])
                                                    ])
                                            ], width=6)
                                        ], className="mt-3")
                                    ])
                                ], style={'flex': 1, 'display': 'flex', 'flexDirection': 'column'})
                            ]
                        )
                    ], width=4, className="d-flex justify-content-center"),

                    dbc.Col([
                        dbc.Card(style={'backgroundColor': FUNDO_ESCURO, 'border': f'2px solid {COR_PRIMARIA}'}, children=[
                            dbc.CardBody([
                                html.H5("Churn por Método de Pagamento", className="card-title", style={'color': COR_PRIMARIA}),
                                dcc.Graph(
                                    figure=px.bar(total_pred_por_tipo_pagamento,
                                                 x='Método_de_Pagamento',
                                                 y='Churn_Predict',
                                                 color='Método_de_Pagamento',
                                                 title='Comparativo por Método de Pagamento',
                                                 color_discrete_sequence=[COR_PRIMARIA, COR_SECUNDARIA, COR_SUCESSO],
                                                 template='plotly_dark')
                                    .update_layout(paper_bgcolor=FUNDO_ESCURO, plot_bgcolor=FUNDO_ESCURO)
                                )
                            ])
                        ])
                    ], width=8)
                ], className='mb-4'),

                # Segunda Linha de Gráficos
                dbc.Row([
                    dbc.Col([
                        dbc.Card(style={'backgroundColor': FUNDO_ESCURO, 'border': f'2px solid {COR_PRIMARIA}'}, children=[
                            dbc.CardBody([
                                html.H5("Top 5 Estados com Mais Previsões", className="card-title", style={'color': COR_PRIMARIA}),
                                dcc.Graph(
                                    figure=px.bar(total_pred_estados_top5,
                                                 x='Estado',
                                                 y='Churn_Predict',
                                                 color='Estado',
                                                 title='Estados com Maior Risco',
                                                 color_discrete_sequence=px.colors.sequential.Purpor_r,
                                                 template='plotly_dark')
                                    .update_layout(paper_bgcolor=FUNDO_ESCURO, plot_bgcolor=FUNDO_ESCURO)
                                )
                            ])
                        ])
                    ], width=12)
                ], className='mb-4'),

                # Terceira Linha de Gráficos
                dbc.Row([
                    dbc.Col([
                        dbc.Card(style={'backgroundColor': FUNDO_ESCURO, 'border': f'2px solid {COR_PRIMARIA}'}, children=[
                            dbc.CardBody([
                                html.H5("Churn por Tipo de Internet", className="card-title", style={'color': COR_PRIMARIA}),
                                dcc.Graph(
                                    figure=px.bar(total_pred_tipo_internet.reset_index(),
                                                 x='Tipo_de_Internet',
                                                 y='Churn_Predict',
                                                 color='Tipo_de_Internet',
                                                 title='Comparativo de Serviço de Internet',
                                                 color_discrete_sequence=[COR_PRIMARIA, COR_SECUNDARIA],
                                                 template='plotly_dark')
                                    .update_layout(paper_bgcolor=FUNDO_ESCURO, plot_bgcolor=FUNDO_ESCURO)
                                )
                            ])
                        ])
                    ], width=6),

                    dbc.Col([
                        dbc.Card(style={'backgroundColor': FUNDO_ESCURO, 'border': f'2px solid {COR_PRIMARIA}'}, children=[
                            dbc.CardBody([
                                html.H5("Distribuição por Faixa Etária", className="card-title", style={'color': COR_PRIMARIA}),
                                dcc.Graph(
                                    figure=px.bar(total_cliente_grupo_idade_pred,
                                                 x='Faixa_Idade',
                                                 y='Total_Clientes',
                                                 category_orders={'Faixa_Idade': ordem_idade},
                                                 title='Distribuição Etária',
                                                 color_discrete_sequence=[COR_PRIMARIA],
                                                 template='plotly_dark')
                                    .update_layout(paper_bgcolor=FUNDO_ESCURO, plot_bgcolor=FUNDO_ESCURO)
                                )
                            ])
                        ])
                    ], width=6)
                ], className='mb-4'),

                # Quarta Linha de Gráficos
                dbc.Row([
                    dbc.Col([
                        dbc.Card(style={'backgroundColor': FUNDO_ESCURO, 'border': f'2px solid {COR_PRIMARIA}'}, children=[
                            dbc.CardBody([
                                html.H5("Distribuição por Tempo de Contrato", className="card-title", style={'color': COR_PRIMARIA}),
                                dcc.Graph(
                                    figure=px.bar(total_clientes_tempo_contratado_pred,
                                                 x='Faixa_Tempo_Contrato',
                                                 y='Total_Clientes',
                                                 category_orders={'Faixa_Tempo_Contrato': ordem_tempo},
                                                 title='Tempo de Contrato Ativo',
                                                 color_discrete_sequence=px.colors.sequential.Purpor_r,
                                                 template='plotly_dark')
                                    .update_layout(paper_bgcolor=FUNDO_ESCURO, plot_bgcolor=FUNDO_ESCURO)
                                )
                            ])
                        ])
                    ], width=12)
                ], className='mb-4')
            ],
            tabClassName="fw-bold"
        )
    ])
], fluid=True, style={
    'backgroundColor': FUNDO_ESCURO,
    'padding': '20px',
    'minHeight': '100vh'
})

if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>